In [15]:
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.search.documents import SearchClient
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from random import randint
from typing import Dict, Tuple
from unidecode import unidecode
import asyncio
import glob
import os.path
import re
search_endpoint = "https://ai-search-acpr-hackathon.search.windows.net" # Add your Azure AI Search endpoint here
search_key = "SEARCH_API_KEY"  # Add your Azure AI Search admin key here
search_index_name = "YOURINDEXNAME" # Add your Azure AI Search index name here

os.environ["SearchEndpoint"] = search_endpoint
os.environ["SearchKey"] = search_key
os.environ['SearchIndex'] = search_index_name

search_endpoint = "https://ai-search-acpr-hackathon.search.windows.net"
search_credential = AzureKeyCredential("SEARCH_API_KEY")
search_client = SearchClient(
  endpoint=search_endpoint,
  index_name="YOURINDEXNAME",
  credential=search_credential,
)

In [16]:
from PyPDF2 import PdfReader 
def pdf_to_markdown(source: str) -> Tuple[str, str]:
  
    # creating a pdf reader object 
    reader = PdfReader(source) 
    
    # printing number of pages in pdf file 
    texte = ''
    for page in range(len(reader.pages)):
    # getting a specific page from the pdf file 
        texte_page = reader.pages[page] 
        texte_page = texte_page.extract_text() 

        texte = texte + texte_page
    #open text file
    label = source.split('.')[0]
    text_file = open(label+".txt", "w")
    #write string to file
    text_file.write(texte)
    #close file
    text_file.close()



doc_results: Dict[str, str] = {}
doc_tasks = []

for source in glob.glob("dataset/*.pdf"):
    pdf_to_markdown(source)



In [17]:
lang = "french"
stemmer = SnowballStemmer(lang)
sentence_r = r"[^\w\s+\-/'\",:;()@=]"


def compress_and_clean(text: str) -> str:
    """
    Compress and clean a text.

    Use the Snowball stemmer to stem words and remove all special characters, as the LLM does not give a damn.
    """
    text = text.replace("\\", "")  # Remove all backslashes
    text = re.sub(r":[a-z]*:", "", text)  # Remove all :unselected: and :selected: tags
    text = re.sub(r"<!--[^<>]*-->", "", text)  # Remove all comments
    tokenized = word_tokenize(text, language=lang)
    tokens = [stemmer.stem(token) for token in tokenized]
    prompt = " ".join(tokens)
    return prompt


def data(content: str, source_uri: str, title: str, iterator: int) -> dict[str, str]:
    """
    Generate a data object for the search index.

    Use deterministic ID to avoid duplicates after a new run. Remove all special characters from title.
    """
    return {
        "content": content,
        "id": f"{'_'.join(re.sub('[^a-z0-9]', ' ', unidecode(source_uri).lower()).split())}-{iterator}",
        "source_uri": unidecode(source_uri).lower(),
        "title": ' '.join(re.sub('[^a-z0-9]', ' ', unidecode(title).lower()).split()),
    }

In [18]:
chuncks = []
iterator = 0
f = open('dataset/guideligne.txt','r')
content = f.read()
# Split the document into sections
for section in content.split("\n#"):
    lines = section.split("\n")
    title = lines[0].strip()
    paragraph = " ".join(lines[1:])
    print(paragraph)
    # Skip empty sections
    if not section:
      continue
    # Split the paragraph into sentences and group them into groups of 300 characters
    groups = []
    group = ""
    separators = re.findall(sentence_r, paragraph)
    splits = re.split(sentence_r, paragraph)
    for i, separator in enumerate(separators):
        sentence = compress_and_clean(splits[i] + separator)
        if len(group) + len(sentence) >= 300:
            groups.append(group)
            group = ""
        group += sentence
    if group:
        groups.append(group)
    # Create a data object for each group
    for group in groups:
        chuncks.append(data(
          f"# {title} {group}",
          source,
          title,
          iterator,
        ))
        iterator += 1

print(chuncks[:5])
print(f"Created {len(chuncks)} chunks")

                          Lignes directrices conjointes de l’ Autorité de contrôle   prudentiel et de résolution et de TRACFIN  sur les obligations de  déclaration et d’information à TRACFIN           Document de nature explicative     (Version actualisée avec mise à jour des dispositions   législatives et réglementaires au 1er octobre  2018 )     Lignes directrices conjointes de l’ Autorité de contrôle prudentiel et de résolution et de Tracfin sur la déclaration de soupçon       2     Contenu     Introduction  ................................ ................................ ................................ ................................ ..........................  4  1 Les obligations de vigilance à l’égard de la clientèle  ................................ ................................ ...............  7  1.1 Les obligations de vigilance dans le cadre des relations d’affaires  ................................ ..................  8  1.1.1 Mise en œuvre des obligations de vigilanc

In [19]:
import_size = 100
import_part = chuncks[:import_size]
while import_part:
    print(import_part)
    print(f"Uploading {len(import_part)} documents to Azure Search")
    search_client.upload_documents(import_part)
    chuncks = chuncks[import_size:]
    import_part = chuncks[:import_size]

print(f"There are {search_client.get_document_count()} documents in the index")

[{'content': '#  lign directric conjoint de l ’autor de contrôl prudentiel et de résolu et de tracfin sur le oblig de déclar et d ’inform à tracfin docu de natur expliqu ( version actualis avec mis à jour de disposit législ et réglementair au 1er octobr 2018 ) lign directric conjoint de l ’', 'id': 'dataset_guideligne_pdf-0', 'source_uri': 'dataset/guideligne.pdf', 'title': ''}, {'content': '#  autor de contrôl prudentiel et de résolu et de tracfin sur la déclar de soupçon 2 contenu introduct ..........................................................................................................................................................4 1 le oblig de vigil à l ’', 'id': 'dataset_guideligne_pdf-1', 'source_uri': 'dataset/guideligne.pdf', 'title': ''}, {'content': '#  égard de la clientel ...............................................................................7 1 .1 le oblig de vigil dan le cadr de relat d ’affair ..................................................8 1 .1 .